YOLOv8 model connected with Anvil Application

In [2]:
import anvil.server
import anvil.media
from ultralytics import YOLO  
from sklearn.cluster import KMeans
from PIL import Image, UnidentifiedImageError
import numpy as np
import webcolors
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import textwrap

# Load the trained model
model = YOLO('/Users/kimberleycollins/Desktop/Thesis/Thesis/Prototype/YOLOv8/YOLOv8 -Training with all images/datasets/runs/detect/train/weights/best.pt')  

# Connect to Anvil server
anvil.server.connect("server_RDZLDM6W6IB7QED6DJDBDZVJ-KZDBRSHH6OZRPUJ6")

def find_dominant_color(image):
    try:
        pixels = image.reshape((-1, 3))
        kmeans = KMeans(n_clusters=1, n_init=10)
        kmeans.fit(pixels)
        dominant_color = kmeans.cluster_centers_.astype(int)[0]
        return '#%02x%02x%02x' % tuple(dominant_color)
    except Exception as e:
        return None

def extract_olive_color(image, bbox):
    try:
        olive_image = image[bbox[1]:bbox[3], bbox[0]:bbox[2]]
        dominant_color = find_dominant_color(olive_image)
        return dominant_color
    except Exception as e:
        return None

def closest_color(requested_color):
    min_colors = {}
    for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - requested_color[0]) ** 2
        gd = (g_c - requested_color[1]) ** 2
        bd = (b_c - requested_color[2]) ** 2
        min_colors[(rd + gd + bd)] = name
    return min_colors[min(min_colors.keys())]

def get_color_name(hex_color):
    try:
        rgb_color = webcolors.hex_to_rgb(hex_color)
        color_name = closest_color(rgb_color)
        return color_name
    except Exception as e:
        return "Unknown color"

def check_olive_color(hex_color):
    try:
        rgb_color = tuple(int(hex_color.lstrip('#')[i:i+2], 16) for i in (0, 2, 4))
        luminance = 0.299 * rgb_color[0] + 0.587 * rgb_color[1] + 0.114 * rgb_color[2]
        
        color_ranges = {
            "unripe": (120, 180, 70),
            "partially_ripe": (90, 110, 50),
            "fully_ripe": (60, 0, 70),
            "overripe": (20, 20, 20)
        }
        
        ripeness = "unknown"
        if rgb_color >= color_ranges["unripe"] and luminance > 150:
            ripeness = "unripe"
        elif rgb_color >= color_ranges["partially_ripe"] and 100 <= luminance <= 150:
            ripeness = "partially ripe"
        elif rgb_color >= color_ranges["fully_ripe"] and 50 <= luminance <= 100:
            ripeness = "fully ripe"
        elif rgb_color <= color_ranges["overripe"] or luminance < 50:
            ripeness = "overripe"
        
        ripeness_messages = {
            "unripe": "The olives are unripe, ideal for pickling or early harvest products. Not ideal for oil extraction yet.",
            "partially ripe": "The olives are partially ripe and may have a more bitter taste, suitable for certain types of processing. Best for harvesting if aiming for early-harvest olive oil.",
            "fully ripe": "The olives appear to be fully ripe and ready for most culinary uses or oil extraction. Ideal time for harvesting for high-quality olive oil.",
            "overripe": "The olives are overripe; they might be too soft or mushy, suitable only for some types of oil or immediate consumption. Immediate harvesting is recommended to avoid spoilage.",
            "unknown": "It's difficult to determine the ripeness of these olives based on the image provided."
        }
        
        return ripeness_messages[ripeness]
    
    except Exception as e:
        return "An error occurred while analyzing olive color."

def suggest_dishes_for_olives(olive_color_message):
    if 'Black olives' in olive_color_message:
        return ("Ideal for bold-flavored dishes like tapenade, puttanesca sauce, Mediterranean pizza, or as a rich topping for Greek salads. Black olives can also be used to enhance the flavor of roast lamb or as part of a cheese and charcuterie board.")
    else:
        return ("Perfect for lighter fare such as martinis, Greek salads, or paired with cheese and wine. Green olives can also be used in chicken tagine, olive bread, or as a garnish for fish dishes.")

def olive_preservation_tips(color):
    if color == 'green':
        tips = ("Green olives can be preserved in brine, vinegar, or olive oil. "
                "Brining is a traditional method that involves soaking the olives in a salt water solution for several weeks, "
                "allowing them to ferment slightly and develop their flavor. "
                "After brining, they can be seasoned with herbs and stored in olive oil or vinegar for enhanced taste.")
    else:
        tips = ("Black olives are often preserved in oil or dry-cured. "
                "Dry-curing involves covering the olives in salt to draw out moisture, resulting in a wrinkled, concentrated flavor. "
                "After curing, they can be rinsed and stored in olive oil with herbs or lemon zest for added flavor. "
                "Storing in oil helps keep them moist and flavorful.")
    return tips

def olive_health_tips():
    tips = (
        "To keep your preserved olives healthy, ensure they are stored in a cool, dark place, away from direct sunlight. "
        "If preserved in liquid (brine, vinegar, or oil), make sure the olives are fully submerged to prevent mold and oxidation. "
        "Containers should be sealed tightly to keep out air and contaminants. "
        "For olives preserved in oil, use sterilized jars and high-quality olive oil to prevent the growth of bacteria, including botulism. "
        "Regularly check your olives for signs of spoilage, such as off-odors, discoloration, or mold. "
        "If preserved in brine, changing the brine every few months can help maintain freshness and flavor. "
        "Always use clean utensils to handle olives to prevent contamination. "
        "Consider refrigerating after opening to prolong their shelf life, especially for olives in liquid preservations."
    )
    return tips

@anvil.server.callable
def classify_image(media):
    try:
        with anvil.media.TempFile(media) as filename:
            try:
                image = Image.open(filename)
            except UnidentifiedImageError:
                return {"error": "Unsupported image format or corrupted file."}
            if image.mode != 'RGB':
                image = image.convert('RGB')
            
            results = model.predict(image)  # Placeholder for actual prediction
            if len(results) == 0:
                return {"error": "No olives detected in the image."}
            
            bbox = [100, 50, 200, 150]  # Placeholder for actual bounding box coordinates
            np_image = np.array(image)
            olive_color = extract_olive_color(np_image, bbox)
            if olive_color is None:
                return {"error": "Failed to find a dominant color in the selected portion of the image."}
            
            olive_color_name = get_color_name(olive_color)
            olive_color_message = check_olive_color(olive_color)
            dish_suggestion = suggest_dishes_for_olives(olive_color_message)
            color = 'green' if 'Green' in olive_color_message else 'black'
            preservation_tips = olive_preservation_tips(color)
            health_tips = olive_health_tips()
            
            return {"olive_color": olive_color, "olive_color_name": olive_color_name, "olive_color_message": olive_color_message, "dish_suggestion": dish_suggestion, "preservation_tips": preservation_tips, "health_tips": health_tips}
    except Exception as e:
        return {"error": f"An unexpected error occurred: {str(e)}"}

@anvil.server.callable
def generate_report(olive_color, olive_color_name, olive_color_message, preservation_tips, health_tips, dish_suggestion, report_date, report_time):
    filename = "/tmp/olive_analysis_report.pdf"
    c = canvas.Canvas(filename, pagesize=letter)
    width, height = letter

    def draw_wrapped_text(c, text, x, y, max_width, line_height):
        lines = textwrap.wrap(text, width=max_width)
        for line in lines:
            c.drawString(x, y, line)
            y -= line_height  # Adjust this value for line spacing

    max_width = 100  # Adjust this value to control text wrapping width
    line_height = 12  # Adjust this value for line spacing

    c.setFont("Helvetica", 14)
    c.drawString(30, height - 50, "Olive Analysis Report")
    c.setFont("Helvetica", 12)
    c.drawString(30, height - 80, f"Olive Color: {olive_color} ({olive_color_name})")
    
    y_position = height - 110
    draw_wrapped_text(c, f"Olive Color Message: {olive_color_message}", 30, y_position, max_width, line_height)
    y_position -= line_height * (len(textwrap.wrap(f"Olive Color Message: {olive_color_message}", width=max_width)) + 1)
    
    draw_wrapped_text(c, f"Preservation Tips: {preservation_tips}", 30, y_position, max_width, line_height)
    y_position -= line_height * (len(textwrap.wrap(f"Preservation Tips: {preservation_tips}", width=max_width)) + 1)
    
    draw_wrapped_text(c, f"Health Tips: {health_tips}", 30, y_position, max_width, line_height)
    y_position -= line_height * (len(textwrap.wrap(f"Health Tips: {health_tips}", width=max_width)) + 1)
    
    draw_wrapped_text(c, f"Suggested Dish: {dish_suggestion}", 30, y_position, max_width, line_height)
    
    # Add date and time to the report
    c.setFont("Helvetica", 10)
    y_position -= line_height * 3
    c.drawString(30, y_position, f"Report Date: {report_date}")
    y_position -= line_height
    c.drawString(30, y_position, f"Report Time: {report_time}")

    c.save()
    return anvil.media.from_file(filename, "application/pdf")


# Keep the server running
anvil.server.wait_forever()



0: 320x608 1 Nocellara del Belice -Italy-, 89.2ms
Speed: 1.9ms preprocess, 89.2ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 608)


KeyboardInterrupt: 